# Using Markov Chain to Generate Miscal Sequences

In [ ]:
! pip install mido
! pip install pretty_midi
! pip install winaudio

## Downloading Dataset
In this block of code, we are downloading a MIDI file named "Gnossienne_1_Saya_Tomoko-s-gnossie1.mid" using the wget command. We "-O" option to specify the output filename as "gnossienne.mid".


Erik Satie's melodies are often characterized by their simplicity and minimalistic structure, which contribute to their accessibility and ease of comprehension.

The simplicity of Satie's melodies makes them particularly suitable for this Markov chain implementation for melody generation.

In [ ]:
! wget "https://websites.umich.edu/~bbowman/midi/satie/Gnossienne_1_Saya_Tomoko-s-gnossie1.mid" -O gnossienne.mid

## Importing Necessary Modules

In [ ]:
import pretty_midi
import IPython.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Loading

We use the pretty_midi library to load a MIDI file named "gnossienne.mid" from the specified path into a PrettyMIDI object named pm. This library allows for easy manipulation and analysis of MIDI data.

In [ ]:
pm = pretty_midi.PrettyMIDI("./gnossienne.mid")

The resulting audio is displayed using IPython.display.Audio, which allows for inline audio playback in Jupyter notebooks.

In [ ]:
fs = 16000
IPython.display.Audio(pm.instruments[0].synthesize(fs=fs), rate=fs)

We create a dataframe with column names "start", "end", "pitch", "velocity" and the values are lists containing the corresponding information extracted from each note.

In [ ]:
notes = pm.instruments[0].notes
df = pd.DataFrame({
    "start" : [note.start for note in notes],
    "end" : [note.end for note in notes],
    "pitch" : [note.pitch for note in notes],
    "velocity" : [note.velocity for note in notes],
})

In [ ]:
df

In [ ]:
print(df)
print(df["pitch"].unique())

This block of code calculates the average interval and velocity between successive notes in a melody, based on the pitch values. This information is organized into a DataFrame, averages_df, presenting a clear overview of the rhythm and velocity patterns between consecutive notes. This analysis aids in ensuring that the generated sequence, using Markov Chains, maintains a rhythmic coherence.

In [ ]:
averages = {}
for i in range(len(df) - 1):
    prev_pitch = df.iloc[i]['pitch']
    curr_pitch = df.iloc[i + 1]['pitch']
    start = df.iloc[i]['start']
    end = df.iloc[i + 1]['end']
    velocity = df.iloc[i]['velocity']

    interval = end - start

    if (prev_pitch, curr_pitch) not in averages:
        averages[(prev_pitch, curr_pitch)] = {'interval': [], 'velocity': []}

    averages[(prev_pitch, curr_pitch)]['interval'].append(interval)
    averages[(prev_pitch, curr_pitch)]['velocity'].append(velocity)

# averages for each combination
averages_table = []
for key, value in averages.items():
    prev_pitch, curr_pitch = key
    avg_interval = sum(value['interval']) / len(value['interval'])
    avg_velocity = sum(value['velocity']) / len(value['velocity'])
    averages_table.append([prev_pitch, curr_pitch, avg_interval, avg_velocity])
averages_df = pd.DataFrame(averages_table, columns=['previous_pitch', 'current_pitch', 'avg_interval', 'avg_velocity'])
averages_df

In [ ]:
averages_df.head(10)

## Design Markov Chain

This block of code calculates a transition matrix representing the frequency of transitions between different pitch values. We extract unique pitch values. It initializes a square matrix with dimensions corresponding to the number of unique pitch values. It iterates through the DataFrame to identify successive pairs of pitches, incrementing the corresponding cell in the transition matrix each time a transition occurs.

In [ ]:
unique_pitches = df["pitch"].unique()
n = len(unique_pitches)
transition_matrix = np.zeros((n, n))

for i in range(len(df) - 1):
    pitch_from = df.iloc[i]["pitch"]
    pitch_to = df.iloc[i + 1]["pitch"]
    transition_matrix[np.where(unique_pitches == pitch_from), np.where(unique_pitches == pitch_to)] += 1

In this code block, a heatmap is being plotted to visualize the transition matrix calculated earlier.

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(transition_matrix, cmap='Blues')

# Add annotations for each cell
for i in range(n):
    for j in range(n):
        plt.text(j, i, '{:.0f}'.format(transition_matrix[i, j]),
                 ha='center', va='center', color='black')

plt.colorbar()
plt.xticks(range(n), unique_pitches, rotation=90)
plt.yticks(range(n), unique_pitches)
plt.title("Matriz de Transición - Frecuencia")
plt.show()

This line of code normalizes the transition matrix along its rows. It divides each element in every row of the transition matrix by the sum of all elements in that row. This normalization ensures that each row of the transition matrix represents a probability distribution, where the sum of probabilities for transitions from a particular pitch to all other pitches adds up to 1.

In [ ]:
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(transition_matrix, cmap='Blues')
# Add annotations for each cell
for i in range(n):
    for j in range(n):
        plt.text(j, i, '{:.2f}'.format(transition_matrix[i, j]),
                 ha='center', va='center', color='black')
plt.colorbar()
plt.xticks(range(n), unique_pitches, rotation=90)
plt.yticks(range(n), unique_pitches)
plt.title("Matriz de Transición - probabilidades")
plt.show()


This block of code uses the scipy.stats module to create a discrete random variable representing the pitch distribution observed in the input music sequence. It calculates the frequency of occurrence for each unique pitch in the sequence and creates a discrete random variable (pitch_rv) with pitch values as possible outcomes and their corresponding probabilities based on their frequencies. This random variable encapsulates the pitch distribution of the original music sequence, allowing for the generation of new sequences that mimic its pitch patterns.

In [ ]:
from scipy.stats import rv_discrete
pitch_counts = df["pitch"].value_counts()
pitch_rv = rv_discrete(values=(unique_pitches, pitch_counts.values / pitch_counts.sum()))

This method generates audio waveforms corresponding to a sequence of musical pitches

In [ ]:
def play_note(pitches, duration=1.0, sample_rate=22050):
    tones = []
    for pitch in pitches:
        freq = 2 ** ((pitch - 49) / 12) * 440
        t = np.linspace(0, duration, int(duration * sample_rate), False)
        tone = np.sin(2 * np.pi * freq * t)
        tones.append(tone)

    audio = np.concatenate(tones)
    return audio, sample_rate

This one creates a PrettyMIDI Instrument object representing a piano and populates it with notes based on a sequence of pitches and average transition information.

In [ ]:
from pretty_midi import Instrument, Note
def play_notes(pitches, df, df_avgs):
  my_instrument = Instrument(program=1, is_drum=False, name="piano")
  counter_start = 0
  current_pitch = pitches[0]
  for p_i in range(1, len(pitches[1:])):
    row = df_avgs[(df_avgs["previous_pitch"]==current_pitch) & (df_avgs["current_pitch"]==pitches[p_i])]
    my_instrument.notes.append(Note(
        velocity=row["avg_velocity"].iloc[0],
        pitch=current_pitch,
        start=counter_start,
        end = start + row["avg_interval"].iloc[0]
    ))
    current_pitch = pitches[p_i]
    counter_start += row["avg_interval"].iloc[0]
  return my_instrument

This block generates a random initial pitch from the pitch distribution and plays it as audio, displaying the pitch as output.

In [ ]:
current_pitch = pitch_rv.rvs()
print(f"Nota inicial: {current_pitch}")
audio, sample_rate = play_note([current_pitch])
IPython.display.Audio(audio, rate=sample_rate)

This code generates a sequence of pitches by iteratively selecting the next pitch based on the transition probabilities encoded in the transition matrix. It starts with an initial pitch randomly chosen from the pitch distribution. Then, for a specified number of transitions, it probabilistically selects the next pitch based on the transition matrix and adds it to the sequence.

- np.where(unique_pitches == current_pitch): finds the index of the current_pitch in the array unique_pitches, returning a tuple with the indices where the condition is met.
- transition_matrix[np.where(unique_pitches == current_pitch)][0]:  retrieves the row of the transition matrix corresponding to the transition probabilities from the current pitch to all other pitches.
- np.random.multinomial(1, transition_matrix[np.where(unique_pitches == current_pitch)][0]): samples from a multinomial distribution defined by the probabilities in the row obtained. Returns an array with zeros. except for one randomly chosen index, which represents the next pitch to transition to.
- .argmax(): returns the index of the maximum value in the array obtained from the previous step, selecting the pitch with the highest transition probability.
- unique_pitches: retrieves the corresponding pitch value from the array unique_pitches based on the index of the maximum transition probability obtained. This is the next pitch in the sequence.

In [ ]:
# generate the sequence of pitches
num_transitions = 30
current_pitch = pitch_rv.rvs()
sequence = [current_pitch]

for _ in range(num_transitions):
    next_pitch = unique_pitches[np.random.multinomial(1, transition_matrix[np.where(unique_pitches == current_pitch)][0]).argmax()]
    sequence.append(next_pitch)
    print(f"Transición: {current_pitch} -> {next_pitch}")
    current_pitch = next_pitch

# Play the sequence
audio, sample_rate = play_note(sequence)
IPython.display.Audio(audio, rate=sample_rate)

This synthesizes audio from the generated Instrument object at a sampling rate of 25000 Hz using the synthesize method and displays it as an audio output

In [ ]:
inst = play_notes(sequence, df, averages_df)
IPython.display.Audio(inst.synthesize(fs=25000), rate=25000)